In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import lifelines
import pycox
import xgboost as xgb
from sklearn.model_selection import train_test_split

from xgbse import XGBSEKaplanNeighbors, XGBSEKaplanTree, XGBSEDebiasedBCE, XGBSEBootstrapEstimator, XGBSEStackedWeibull
from xgbse.converters import convert_data_to_xgb_format, convert_to_structured, convert_y
from xgbse.non_parametric import get_time_bins

from benchmark import BenchmarkLifelines, BenchmarkXGBoost, BenchmarkXGBSE, BenchmarkPysurvival
#from pysurvival.models.survival_forest import ConditionalSurvivalForestModel

# setting seed
np.random.seed(42)

In [3]:
def split_train_test_valid(dataf, test_size=.2, valid_size=.01, random_state=1):
    df_train, df_test = train_test_split(dataf, test_size=test_size, random_state=random_state)
    df_train, df_valid = train_test_split(df_train, test_size=valid_size, random_state=random_state)
    return df_train, df_valid,  df_test

## Data

In [4]:
from pycox.datasets import sac3
sac3_df = sac3.read_df()
sac3_df.drop(["duration_true", "event_true", "censoring_true"], axis=1, inplace=True)

df_train, df_valid, df_test = split_train_test_valid(sac3_df)

T_train = df_train.duration
E_train = df_train.event
TIME_BINS = time_bins = get_time_bins(T_train, E_train, size=30)

Dataset 'sac3' not created yet. Making dataset...
Done


## Model

Let us fit a model and check performance.

In [5]:
results = pd.DataFrame()

## Lifelines

In [6]:

bench_lifelines = BenchmarkLifelines(lifelines.CoxPHFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Cox-PH"
)

In [7]:
bench_lifelines.train()

In [8]:
cox_lifelines_results = bench_lifelines.test()
cox_lifelines_results

{'model': 'Cox-PH',
 'c-index': 0.6821608577751815,
 'ibs': 0.16538314664366988,
 'dcal_pval': 2.960248829034621e-63,
 'dcal_max_dev': 0.035299304270430085,
 'training_time': 1.839928150177002,
 'inference_time': 0.03926515579223633}

In [9]:
results = results.append(cox_lifelines_results, ignore_index=True)

In [10]:
bench_lifelines = BenchmarkLifelines(lifelines.WeibullAFTFitter(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "Weibull AFT"
)

In [11]:
bench_lifelines.train()
weibull_aft_results = bench_lifelines.test()
weibull_aft_results

{'model': 'Weibull AFT',
 'c-index': 0.682028716612183,
 'ibs': 0.16549935329896318,
 'dcal_pval': 8.433756927103006e-82,
 'dcal_max_dev': 0.03897261022924145,
 'training_time': 2.307232141494751,
 'inference_time': 0.04311800003051758}

In [12]:
results = results.append(weibull_aft_results, ignore_index=True)

## XGBoost

In [13]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - AFT",
        "survival:aft",
)

In [14]:
bench_xgboost.train()
xgboost_aft_results = bench_xgboost.test()
xgboost_aft_results

{'model': 'XGB - AFT',
 'c-index': 0.6906307432431247,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 7.4134063720703125,
 'inference_time': 0.0035758018493652344}

In [15]:
results = results.append(xgboost_aft_results, ignore_index=True)

In [16]:
bench_xgboost = BenchmarkXGBoost(xgb,
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGB - Cox",
        "survival:cox",
)
bench_xgboost.train()
xgboost_cox_results = bench_xgboost.test()
xgboost_cox_results

{'model': 'XGB - Cox',
 'c-index': 0.6861246916297403,
 'ibs': nan,
 'dcal_pval': nan,
 'dcal_max_dev': nan,
 'training_time': 4.885499000549316,
 'inference_time': 0.0024073123931884766}

In [17]:
results = results.append(xgboost_cox_results, ignore_index=True)

## XGB SE

In [18]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEDebiasedBCE(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Debiased BCE",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bce_results = bench_xgboost_embedding.test()
xgboost_bce_results

{'model': 'XGBSE - Debiased BCE',
 'c-index': 0.6898556886301602,
 'ibs': 0.16919460965331368,
 'dcal_pval': 6.341369205743656e-120,
 'dcal_max_dev': 0.044509411533409995,
 'training_time': 47.81409811973572,
 'inference_time': 1.1414539813995361}

In [19]:
results = results.append(xgboost_bce_results, ignore_index=True)

In [20]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanNeighbors(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Neighbors",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kn_results = bench_xgboost_embedding.test()
xgboost_kn_results

{'model': 'XGBSE - Kaplan Neighbors',
 'c-index': 0.6663894784009747,
 'ibs': 0.17540358380688037,
 'dcal_pval': 2.553451853967165e-66,
 'dcal_max_dev': 0.03725501925052292,
 'training_time': 36.75927400588989,
 'inference_time': 416.381933927536}

In [21]:
results = results.append(xgboost_kn_results, ignore_index=True)

In [22]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEKaplanTree(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Kaplan Tree",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_kt_results = bench_xgboost_embedding.test()
xgboost_kt_results

{'model': 'XGBSE - Kaplan Tree',
 'c-index': 0.6309520608231479,
 'ibs': 0.1914181890695273,
 'dcal_pval': 6.061067356778973e-56,
 'dcal_max_dev': 0.033688866155989705,
 'training_time': 1.477849006652832,
 'inference_time': 0.0358281135559082}

In [23]:
results = results.append(xgboost_kt_results, ignore_index=True)

In [24]:
base_tree = XGBSEKaplanTree()

bench_xgboost_embedding = BenchmarkXGBSE(
    XGBSEBootstrapEstimator(base_tree, n_estimators=100),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Bootstrap Trees",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_bootstrap_results = bench_xgboost_embedding.test()
xgboost_bootstrap_results

{'model': 'XGBSE - Bootstrap Trees',
 'c-index': 0.6769903585433326,
 'ibs': 0.16786412434551737,
 'dcal_pval': 1.6511792922848247e-98,
 'dcal_max_dev': 0.043015745266438726,
 'training_time': 164.17341589927673,
 'inference_time': 3.1336629390716553}

In [25]:
results = results.append(xgboost_bootstrap_results, ignore_index=True)

In [26]:
bench_xgboost_embedding = BenchmarkXGBSE(XGBSEStackedWeibull(),
        df_train,
        df_valid,
        df_test,
        "event",
        "duration",
        TIME_BINS,
        "XGBSE - Stacked Weibull",
        "survival:aft",
)
bench_xgboost_embedding.train()
xgboost_weibull_results = bench_xgboost_embedding.test()
xgboost_weibull_results

{'model': 'XGBSE - Stacked Weibull',
 'c-index': 0.6965210329780931,
 'ibs': 0.17058402978933754,
 'dcal_pval': 1.8884687501353905e-76,
 'dcal_max_dev': 0.03968637740815539,
 'training_time': 32.46857404708862,
 'inference_time': 0.152846097946167}

In [27]:
results = results.append(xgboost_weibull_results, ignore_index=True)

In [28]:
results.sort_values("c-index", ascending=False).round(3)

,c-index,dcal_max_dev,dcal_pval,ibs,inference_time,model,training_time
8,0.697,0.040,0.0,0.171,0.153,XGBSE - Stacked Weibull,32.469
2,0.691,NaN,NaN,NaN,0.004,XGB - AFT,7.413
4,0.690,0.045,0.0,0.169,1.141,XGBSE - Debiased BCE,47.814
3,0.686,NaN,NaN,NaN,0.002,XGB - Cox,4.885
0,0.682,0.035,0.0,0.165,0.039,Cox-PH,1.840
1,0.682,0.039,0.0,0.165,0.043,Weibull AFT,2.307
7,0.677,0.043,0.0,0.168,3.134,XGBSE - Bootstrap Trees,164.173
5,0.666,0.037,0.0,0.175,416.382,XGBSE - Kaplan Neighbors,36.759
6,0.631,0.034,0.0,0.191,0.036,XGBSE - Kaplan Tree,1.478


In [29]:
results.sort_values("c-index", ascending=False).round(3).set_index('model').to_markdown()

'| model                    |   c-index |   dcal_max_dev |   dcal_pval |     ibs |   inference_time |   training_time |\n|:-------------------------|----------:|---------------:|------------:|--------:|-----------------:|----------------:|\n| XGBSE - Stacked Weibull  |     0.697 |          0.04  |           0 |   0.171 |            0.153 |          32.469 |\n| XGB - AFT                |     0.691 |        nan     |         nan | nan     |            0.004 |           7.413 |\n| XGBSE - Debiased BCE     |     0.69  |          0.045 |           0 |   0.169 |            1.141 |          47.814 |\n| XGB - Cox                |     0.686 |        nan     |         nan | nan     |            0.002 |           4.885 |\n| Cox-PH                   |     0.682 |          0.035 |           0 |   0.165 |            0.039 |           1.84  |\n| Weibull AFT              |     0.682 |          0.039 |           0 |   0.165 |            0.043 |           2.307 |\n| XGBSE - Bootstrap Trees  |     0.677 